## Load Models and Tokenizers

In [1]:
from tensorflow.python.keras.models import load_model

### Encoder and Decoder Model

In [20]:
encoder_model = load_model('G:/UpX Academy/Deep Learning/Week6SeqToSeq/seq2seq_encoder_eng_hin.hd5')

In [21]:
decoder_model = load_model('G:/UpX Academy/Deep Learning/Week6SeqToSeq/seq2seq_decoder_eng_hin.hd5')

### Load Tokenizers

In [5]:
import pickle

In [6]:
encoder_t = pickle.load(open('encoder_tokenizer_eng','rb'))

In [7]:
decoder_t = pickle.load(open('decoder_tokenizer_hin','rb'))

### Define configuration parameters

In [8]:
max_encoder_seq_length = 22
max_decoder_seq_length = 27

In [23]:
#dictionary to convert decoder output into hindi words
int_to_word_decoder = dict((i,c) for c, i in decoder_t.word_index.items())

### Function to generate padded sequences for Input strings

In [10]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [18]:
def encode_input(input_str):
    
    #convert words to indexes
    encoder_seq = encoder_t.texts_to_sequences([input_str])
    
    #pad sequences
    encoder_input_data = pad_sequences(encoder_seq, maxlen = max_encoder_seq_length,padding = 'post')
    
    return encoder_input_data

## Prediction function

In [12]:
import numpy as np

In [15]:
def decode_sentence(input_str):
    
    #convert input string to padded sequence
    input_seq = encode_input(input_str)
    
    #Get the encoder state values
    decoder_initial_states_value = encoder_model.predict(input_seq)
    
    #Build a sequence with '<start>' - starting sequence for Decoder
    target_seq = np.zeros((1,1))    
    target_seq[0][0] = decoder_t.word_index['<start>']
    
    #flag to check if prediction should be stopped
    stop_loop = False
    
    #Initialize predicted sentence
    predicted_sentence = ''
    
    #start the loop
    while not stop_loop:
        
        predicted_outputs, h, c = decoder_model.predict([target_seq] + 
                                                        decoder_initial_states_value)
        
        #Get the predicted output with highest probability
        predicted_output = np.argmax(predicted_outputs[0,-1,:])
        
        #Get the predicted word from predicter integer
        predicted_word = int_to_word_decoder[predicted_output]
        
        #Check if prediction should stop
        if(predicted_word == '<end>' or len(predicted_sentence) > max_decoder_seq_length):
            
            stop_loop = True
            continue
                    
        #Updated predicted sentence
        if (len(predicted_sentence) == 0):
            predicted_sentence = predicted_word
        else:
            predicted_sentence = predicted_sentence + ' ' + predicted_word
            
        #Update target_seq to be the predicted word index
        target_seq[0][0] = predicted_output
        
        #Update initial states value for decoder
        decoder_initial_states_value = [h,c]
        
    
    return predicted_sentence

In [25]:
if __name__ == '__main__':
    print(decode_sentence('how are you?'))

आप कैसे हो
